In [6]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot
from pandapower.control import ConstControl
from pandapower.timeseries import DFData, OutputWriter, run_timeseries
import matplotlib.pyplot as plt
from scipy.stats import norm

In [12]:
# Define criteria thresholds
voltage_threshold = 0.95  # Minimum acceptable voltage in p.u.
line_loading_threshold = 100.0  # Maximum acceptable line loading in percentage

# Perform base case power flow
pp.runpp(net)
# Check the base case results
print("Base Case: Power flow successfully converged.")
base_voltages = net.res_bus['vm_pu']
base_line_loadings = net.res_line['loading_percent']

Base Case: Power flow successfully converged.


In [13]:
# List of lines to consider for contingencies
line_contingencies = net.line.index.tolist()

# List of generators to consider for contingencies
gen_contingencies = net.gen.index.tolist()

In [14]:
# Results storage
contingency_results = []

# Loop through line contingencies
for line in line_contingencies:
    # Backup the original line state
    original_in_service = net.line.at[line, 'in_service']
    
    # Simulate the contingency by opening the line
    net.line.at[line, 'in_service'] = False
    
    try:
        # Run power flow
        pp.runpp(net)
        
        # Check system stability (voltage and line loading limits)
        max_voltage = net.res_bus['vm_pu'].max()
        min_voltage = net.res_bus['vm_pu'].min()
        max_loading = net.res_line['loading_percent'].max()
        
        contingency_results.append({
            'Contingency': f'Line {line}',
            'Max Voltage (p.u.)': max_voltage,
            'Min Voltage (p.u.)': min_voltage,
            'Max Line Loading (%)': max_loading,
            'Status': 'Stable' if max_voltage < 1.1 and min_voltage > 0.9 and max_loading < 100 else 'Unstable'
        })
    except pp.LoadflowNotConverged:
        contingency_results.append({
            'Contingency': f'Line {line}',
            'Max Voltage (p.u.)': None,
            'Min Voltage (p.u.)': None,
            'Max Line Loading (%)': None,
            'Status': 'Failed to converge'
        })
    
    # Restore the original line state
    net.line.at[line, 'in_service'] = original_in_service

# Loop through generator contingencies
for gen in gen_contingencies:
    # Backup the original generator state
    original_in_service = net.gen.at[gen, 'in_service']
    
    # Simulate the contingency by shutting down the generator
    net.gen.at[gen, 'in_service'] = False
    
    try:
        # Run power flow
        pp.runpp(net)
        
        # Check system stability (voltage and line loading limits)
        max_voltage = net.res_bus['vm_pu'].max()
        min_voltage = net.res_bus['vm_pu'].min()
        max_loading = net.res_line['loading_percent'].max()
        
        contingency_results.append({
            'Contingency': f'Generator {gen}',
            'Max Voltage (p.u.)': max_voltage,
            'Min Voltage (p.u.)': min_voltage,
            'Max Line Loading (%)': max_loading,
            'Status': 'Stable' if max_voltage < 1.1 and min_voltage > 0.9 and max_loading < 100 else 'Unstable'
        })
    except pp.LoadflowNotConverged:
        contingency_results.append({
            'Contingency': f'Generator {gen}',
            'Max Voltage (p.u.)': None,
            'Min Voltage (p.u.)': None,
            'Max Line Loading (%)': None,
            'Status': 'Failed to converge'
        })
    
    # Restore the original generator state
    net.gen.at[gen, 'in_service'] = original_in_service


In [15]:
df_results = pd.DataFrame(contingency_results)
df_results.to_excel("n1_contingency_results.xlsx", index=False)
print("Contingency results saved to 'n1_contingency_results.xlsx'.")

Contingency results saved to 'n1_contingency_results.xlsx'.
